In [194]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [195]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../'))

In [196]:
import json
import pandas as pd
import warnings

from numpy import random
from collections import defaultdict
from functools import reduce

from model.forest import train_forest
from utils import get_importances

DEFAULT_RANDOM_SEED = 774
random.mtrand._rand.seed(DEFAULT_RANDOM_SEED)
seed_list = list(random.random_integers(low=0, high=2**32 - 1, size=50))
warnings.filterwarnings("ignore")

In [197]:
category = "min_tpm_5"
data = pd.read_csv(f"../../preprocessed/{category}/genes.csv").drop(columns=["sample_id"])
subtypes = set(data["subtype"])

In [198]:
def choose_important_genes(data: pd.DataFrame, scoring: str):
  random.mtrand._rand.seed(DEFAULT_RANDOM_SEED)
  forest_params = {
    "class_weight": "balanced"
  }

  search_params = {"n_estimators": (2, 4, 8, 16, 32, 64), "max_depth": (2, 4, 16, 32, None)}
  importances_by_sex_subtype = defaultdict(list)

  for sex in ["Male", "Female"]:
    filtered_sex_dataset = data[data["sex"] == sex]
    for subtype in subtypes:
      targeted_data = filtered_sex_dataset.copy()
      targeted_data["subtype_target"] = targeted_data["subtype"] == subtype

      for seed in seed_list:
        random.mtrand._rand.seed(seed)
        forest_params["random_state"] = seed
        response = train_forest("subtype_target", data=targeted_data, grid_search_scoring=scoring, grid_search_params=search_params, **forest_params)

        importances_target = get_importances(response.model.feature_importances_, response.feature_names, top=None)
        importances_by_sex_subtype[(sex, subtype)].append(importances_target)

  result = defaultdict(dict)
  for key, importances_list in importances_by_sex_subtype.items():
    importances_series: pd.Series = reduce(lambda x, y: x + y, importances_list) / float(len(importances_list))
    importances = [{ "gene": k, "importance": v } for k, v in importances_series.items() if v != 0]
    result[key[1]] |= { key[0]: sorted(importances, key=lambda x: x["importance"], reverse=True) }

  return result

In [ ]:
result = choose_important_genes(data, scoring="recall_micro")
open(f"../../preprocessed/{category}/important_genes_random_forest_recall.json", "w").write(json.dumps(result))

In [200]:
result = choose_important_genes(data, scoring="f1_weighted")
open(f"../../preprocessed/{category}/important_genes_random_forest_f1.json", "w").write(json.dumps(result))

2799796